In [27]:
import os
import glob
import shutil

fail = 0
restart_lines = []
old_out_dir = 'old_failed'
restart_file = '3_SUBMIT_twotierEMI_3class_restart_jobs.sh'

if not os.path.exists(old_out_dir):
    os.makedirs(old_out_dir)

out_files = glob.glob('old_failed/slurm-*.out')
    
for file in out_files:
    f = open(file, "r")
    last_line = f.readlines()[-1]
    if 'Segmentation fault' in last_line:
        fail += 1
        failed_file = last_line.split(':')[0]
        job = "sbatch -t 1-0 --export=filename=" + failed_file + " batch_job.sbatch"
        restart_lines.append(job)
        restart_lines.append("sleep 60")
        shutil.move(os.path.join('/scratch/sk7898/EdgeML/slurm_hpc', file),\
                    os.path.join(old_out_dir, file))  
        
with open(restart_file,'w') as f:
    f.write('\n'.join(restart_lines))